# Applied Machine Learning Homework 4

## Predicting wine quality from review texts and other properties of the wine
### https://www.kaggle.com/zynicide/wine-reviews

Due: Wed 04/22/20 1pm

Alberto Munguia Cisneros

UNI: am5334

## Task 3 Transformers

### Model fit with BERT Transformers, the model was fitted in Google Colab using imputs from Task 1 and 2

In [23]:
# verify GPU availability
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install transformers

import torch
from transformers import pipeline

In [0]:
!pip install category_encoders

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import timeit

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, ShuffleSplit, RepeatedStratifiedKFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, RidgeCV
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, LabelEncoder
from category_encoders import TargetEncoder, LeaveOneOutEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
from sklearn.utils import shuffle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import Normalizer

from scipy.sparse import hstack


In [26]:
from google.colab import files
uploaded = files.upload()


Saving X_test_s.csv to X_test_s (6).csv
Saving X_train_s.csv to X_train_s (6).csv
Saving y_test.csv to y_test (6).csv
Saving y_train.csv to y_train (6).csv


In [0]:
import io
X_train_s = pd.read_csv(io.BytesIO(uploaded['X_train_s.csv']))
X_test_s = pd.read_csv(io.BytesIO(uploaded['X_test_s.csv']))
y_test = pd.read_csv(io.BytesIO(uploaded['y_test.csv']))
y_train = pd.read_csv(io.BytesIO(uploaded['y_train.csv']))


In [28]:
nlp = pipeline("feature-extraction")

In [0]:
K_train=min(10000,X_train_s.shape[0]) # size of proof

docs_train=[]

for d in X_train_s.txt[0:K_train]:
  a = nlp(d)
  a = np.asarray(a)
  new_a = a.reshape(a.shape[1],a.shape[2])
  new_a = np.mean(new_a, axis=0)
  docs_train.append(new_a)


In [0]:
K_test=min(10000,X_test_s.shape[0]) # size of proof

docs_test=[]

for d in X_test_s.txt[0:K_test]:
  a = nlp(d)
  a = np.asarray(a)
  new_a = a.reshape(a.shape[1],a.shape[2])
  new_a = np.mean(new_a, axis=0)
  docs_test.append(new_a)

In [0]:
X_train_BERT = np.vstack(docs_train)
X_test_BERT = np.vstack(docs_test)

In [32]:
ss = ShuffleSplit(n_splits=5)
ridge = Ridge()

res = cross_validate(ridge,X_train_BERT, y_train[0:K_train], scoring=['r2', 'neg_mean_squared_error','neg_mean_absolute_error',
                                                         'explained_variance'], return_train_score=False, cv=ss)
pd.DataFrame(res)

,fit_time,score_time,test_r2,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_explained_variance
0,0.177131,0.005261,0.659319,-3.514643,-1.481143,0.659332
1,0.175930,0.004816,0.652488,-3.335686,-1.434872,0.652529
2,0.189047,0.004949,0.637182,-3.334795,-1.478506,0.637183
3,0.204742,0.004911,0.615954,-3.743423,-1.547589,0.617196
4,0.195050,0.005347,0.649736,-3.458236,-1.461733,0.649894


In [33]:
ridge.fit(X_train_BERT, y_train[0:K_train])
test_score = ridge.score(X_test_BERT, y_test[0:K_test])
print('Average R^2 in Cross_validation : {:.5f}'.format(res['test_r2'].mean()))
print('R^2 in the Test set : {:.5f}'.format(test_score))

Average R^2 in Cross_validation : 0.64294
R^2 in the Test set : 0.67866


In [34]:
ridge = Ridge(max_iter=5000)

ridge_param_grid = {'alpha': np.logspace(-4, 5, 20) }
ridge_gcv = GridSearchCV(ridge, param_grid = ridge_param_grid ,cv=ss, return_train_score=True)
ridge_gcv.fit(X_train_BERT, y_train[0:K_train])
print('Ridge best parameters: {}'.format(ridge_gcv.best_params_))
print('Ridge best R^2: {:.5f}'.format(ridge_gcv.best_score_))

Ridge best parameters: {'alpha': 1.8329807108324339}
Ridge best R^2: 0.64888


In [35]:
test_accuray_ridge = ridge_gcv.best_estimator_.score(X_test_BERT, y_test[0:K_test])
print('R^2 in the Test set : {:.5f}'.format(test_accuray_ridge))

R^2 in the Test set : 0.67762


**From the results above we can conclude that this untuned version of BERT is not better than a simple procedure of BOW in terms of $R^2$. However, it is important to notice that this result could be affected by our choice of the procedure to flatten the tensor issued from the pipeline procedure.**

**Also if we compare with the complete model we observe that the model fit with BERT Transformers is inferior in terms of $R^2$**

